<a href="https://colab.research.google.com/github/Pulkit4761/Projects/blob/main/Mistral.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import streamlit as st
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers
from streamlit_chat import message
import time

def get_pdf_text(pdf_docs):
    text = ""
    for pdf in pdf_docs:
        pdf_reader = PdfReader(pdf)
        for page in pdf_reader.pages:
            text += page.extract_text()
    return text

def get_text_chunks(text):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=50)
    chunks = text_splitter.split_text(text)
    return chunks

def get_vector_store(text_chunks):
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    vector_store = FAISS.from_texts(text_chunks, embedding=embeddings)
    vector_store.save_local("faiss_index")

def get_conversational_chain():
    prompt_template = """
    Answer the question as detailed as possible from the provided context, make sure to provide all the details, if the answer is not in
    provided context just say, "answer is not available in the context", don't provide the wrong answer\n\n
    Context:\n {context}?\n
    Question: \n{question}\n
    Answer:
    """

    model = CTransformers(
        model=r"C:\Users\91767\Downloads\mistral-7b-instruct-v0.1.Q4_K_M.gguf",
        model_type="mistral",
        config={'max_new_tokens': 1024,
                'temperature': 0.5,
                'context_length': 4096}
    )

    prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])
    chain = load_qa_chain(model, chain_type="stuff", prompt=prompt)
    return chain

def handle_user_input(user_question):
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    new_db = FAISS.load_local("faiss_index", embeddings, allow_dangerous_deserialization=True)
    docs = new_db.similarity_search(user_question)
    chain = get_conversational_chain()

    response = chain.run(input_documents=docs, question=user_question)
    return response  # Return the response instead of using st.write()

def main():
    st.set_page_config(
        page_title="Chat PDF",
        layout="wide",
        initial_sidebar_state="expanded"
    )

    # Initialize session state
    if "messages" not in st.session_state:
        st.session_state.messages = []
    if "vector_store_created" not in st.session_state:
        st.session_state.vector_store_created = False

    # Sidebar
    with st.sidebar:
        st.title("Menu:")
        with st.container():
            pdf_docs = st.file_uploader(
                "Upload your PDF Files",
                accept_multiple_files=True,
                help="Select one or more PDF files to process"
            )
            if st.button("Submit & Process", use_container_width=True):
                with st.spinner("Processing..."):
                    raw_text = get_pdf_text(pdf_docs)
                    text_chunks = get_text_chunks(raw_text)
                    get_vector_store(text_chunks)
                    st.session_state.vector_store_created = True
                    st.success("PDFs processed successfully!")

        st.divider()

        if st.button("Clear Chat", use_container_width=True):
            st.session_state.messages = []
            st.rerun()

    # Create a container for the header
    header_container = st.container()
    with header_container:
        st.title("Chat with PDF 🤖")

    # Create a container for chat messages
    chat_container = st.container()
    with chat_container:
        st.markdown("<div style='height: 20px'></div>", unsafe_allow_html=True)

        # Display chat messages
        for i, msg in enumerate(st.session_state.messages):
            message(
                msg["content"],
                is_user=msg["role"] == "user",
                key=f"message_{i}"
            )

        st.markdown("<div style='height: 100px'></div>", unsafe_allow_html=True)

    # Create a container for the chat input
    input_container = st.container()

    # Apply custom CSS
    st.markdown("""
        <style>
        .css-1j5xypr {
            background-color: #f0f2f6;
        }

        .input-container {
            position: fixed;
            bottom: 0;
            left: 50%;
            transform: translateX(-50%);
            width: 100%;
            max-width: 800px;
            padding: 20px;
            background-color: white;
            border-top: 1px solid #ddd;
            z-index: 100;
        }

        .chat-container {
            margin-bottom: 100px;
            padding-bottom: 100px;
        }

        .stChatFloatingInputContainer {
            bottom: 20px;
            background-color: white;
            padding: 20px;
            border-top: 1px solid #ddd;
            z-index: 100;
        }

        .main {
            padding-bottom: 100px;
        }

        .stChatMessage {
            margin-bottom: 20px;
        }
        </style>
    """, unsafe_allow_html=True)

    # Input box at the bottom
    with input_container:
        prompt = st.chat_input("Ask a question about your PDFs...", key="user_input")
        if prompt:
            if not st.session_state.vector_store_created:
                st.warning("Please upload and process PDF files first!")
                return

            # Add user message to chat history
            st.session_state.messages.append({"role": "user", "content": prompt})

            # Show typing indicator
            with st.spinner("Thinking..."):
                # Get response using the PDF processing logic
                response = handle_user_input(prompt)
                time.sleep(0.5)

            # Add assistant response to chat history
            st.session_state.messages.append({"role": "assistant", "content": response})

            # Rerun to update the chat display
            st.rerun()

if __name__ == "__main__":
    main()